**Creación de un dataset que integre todos los datos con los que podrán trabajar los modelos. **

Incluirá todos las variables de caracter financiero así como algunos datos de empresa, invariables al año. Será normalizado y los valores nulos que aún pudieran quedar  se imputrán.


In [118]:
import pandas as pd
#from google.colab import drive
import numpy as np
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV,cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout
from sklearn.metrics import confusion_matrix,roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBClassifier,plot_importance
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE

from sklearn.decomposition import PCA


from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier

from sklearn.impute import SimpleImputer
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

**1 Creación de dataset combinado**

Contienen todas las variables financieras de cada año. Es una fusió horizontal de todos los datasets preprocesados a partir de las hojas de calculo de cada variable financiera.

Este dataset se usará para entrenar modelos donde todas las variables predictoras para una empresa se combinan en un mismo registro que engloba todos los años.

También se usará  para crear a partir de él un array tridimensional ( empresa, paso temporal -año-, features) para entrenar redes neuronales como LTSM.



In [119]:
ruta="../data/"



# Se cargan los datasets preprocesados correspondientes a cada variable financiera, donde cada fila es una empresa y cada columna un año


df_endeudamiento = pd.read_csv(ruta+'endeudamientoProcesado.csv')
df_solvencia = pd.read_csv(ruta+'solvenciaProcesado.csv')
df_patrimonio = pd.read_csv(ruta+'patrimonioProcesado.csv')

df_gfinancieros = pd.read_csv(ruta+'gfinancierosProcesado.csv')
df_fmaniobra = pd.read_csv(ruta+'fmaniobraProcesado.csv')
df_resultados= pd.read_csv(ruta+'resultadosProcesado.csv')
df_activos= pd.read_csv(ruta+'activosProcesado.csv')
df_ventas= pd.read_csv(ruta+'ventasProcesado.csv')


# También se incluyen los datasets preprocesados  de los índices zombi y un subcojunto de variables de empresa

df_zombi = pd.read_csv(ruta+'zombiProcesado.csv')

df_datosEmpresa= pd.read_csv(ruta+'datosEmpresaProcesado.csv')

In [120]:
def escalar_dataframe(df):

    # Los valores de cada dataframe se normalizan entre 0 y 1
    # Los valores nan que no hayan sido imputados todaía (filas con todos sus valores nulos) se sustituyen por la media del dataset


    cols = [col for col in df.columns if col != "ID"]
    scaler = MinMaxScaler()
    df[cols] = scaler.fit_transform(df[cols])

    imputer = SimpleImputer(strategy='mean')
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns, index=df.index)



    return df

In [121]:
# Normalización de todos los datsets leídos

df_endeudamiento= escalar_dataframe(df_endeudamiento)
df_solvencia= escalar_dataframe(df_solvencia)
df_patrimonio= escalar_dataframe(df_patrimonio)
df_gfinancieros= escalar_dataframe(df_gfinancieros)
df_fmaniobra= escalar_dataframe(df_fmaniobra)
df_resultados= escalar_dataframe(df_resultados)
df_activos= escalar_dataframe(df_activos)
df_ventas= escalar_dataframe(df_ventas)
df_datosEmpresa= escalar_dataframe(df_datosEmpresa)

In [122]:
# Unir los datasets en un solo DataFrame utilizando el ID de empresa como clave
# Se trata de join horizontal.
# Cada fila tendra la estructura:

#ID Solvencia21010... Solvencia2022, AC2010...AC2022, EDAD_EMPRESA... Zombi2010...Zombi2022

df_combined = pd.merge(df_endeudamiento, df_solvencia, on="ID", how="inner")
df_combined = pd.merge(df_combined, df_patrimonio, on="ID", how="inner")
df_combined = pd.merge(df_combined, df_gfinancieros, on="ID", how="inner")
df_combined = pd.merge(df_combined, df_fmaniobra, on="ID", how="inner")
df_combined = pd.merge(df_combined, df_resultados, on="ID", how="inner")
df_combined = pd.merge(df_combined, df_activos, on="ID", how="inner")
df_combined = pd.merge(df_combined, df_zombi, on="ID", how="inner")
df_combined = pd.merge(df_combined, df_ventas, on="ID", how="inner")
#df_combined = pd.merge(df_combined, df_datosEmpresa, on="ID", how="inner")

In [123]:
df_combined.head(15)

,ID,Endeudamiento2010,Endeudamiento2011,Endeudamiento2012,Endeudamiento2013,Endeudamiento2014,Endeudamiento2015,Endeudamiento2016,Endeudamiento2017,Endeudamiento2018,...,RAIE2013,RAIE2014,RAIE2015,RAIE2016,RAIE2017,RAIE2018,RAIE2019,RAIE2020,RAIE2021,RAIE2022
0,233151.0,0.453292,0.463274,0.488989,0.507304,0.481316,0.475423,0.478861,0.437031,0.434540,...,0.056812,0.046632,0.021428,0.017839,0.020914,0.029336,0.021016,0.019432,0.025093,0.033009
1,233207.0,0.431092,0.440383,0.452196,0.466782,0.473426,0.475448,0.467494,0.427205,0.427688,...,0.010667,0.010297,0.012274,0.013369,0.013406,0.028016,0.017235,0.014639,0.022214,0.027376
2,233235.0,0.483459,0.485459,0.502175,0.861161,0.653494,0.469362,0.457144,0.420085,0.418667,...,0.000153,0.000250,0.000697,0.002088,0.002114,0.014591,0.002264,0.002940,0.003009,0.003443
3,233284.0,0.446207,0.435467,0.453304,0.471591,0.481728,0.492106,0.479937,0.442631,0.440994,...,0.157557,0.153629,0.180860,0.209404,0.197542,0.205397,0.212179,0.211643,0.298349,0.325857
4,233291.0,0.438427,0.434249,0.451011,0.468787,0.474433,0.473789,0.473166,0.434365,0.435842,...,0.095421,0.094723,0.108792,0.104641,0.084273,0.097916,0.102741,0.103662,0.136250,0.177452
5,233382.0,0.423547,0.429093,0.447022,0.464949,0.471447,0.470922,0.459908,0.421044,0.417966,...,0.000514,0.002016,0.001446,0.000784,0.001710,0.013691,0.013443,0.001489,0.001547,0.001807
6,233417.0,0.423544,0.424461,0.442522,0.460795,0.469525,0.469136,0.458773,0.420584,0.421154,...,0.002670,0.002737,0.002753,0.002749,0.004226,0.016420,0.003917,0.003788,0.003707,0.003783
7,233487.0,0.459389,0.466973,0.467298,0.494413,0.497998,0.523713,0.495139,0.459328,0.447648,...,0.040635,0.043652,0.047924,0.051489,0.053039,0.068278,0.061820,0.058302,0.070618,0.084941
8,233529.0,0.422424,0.430586,0.443272,0.460358,0.472161,0.466311,0.456018,0.418426,0.418274,...,0.000999,0.001583,0.000041,0.001970,0.001342,0.014585,0.000374,0.000193,0.000414,0.000063
9,233550.0,0.436800,0.431641,0.448165,0.465553,0.471223,0.471518,0.473134,0.437864,0.436178,...,0.053051,0.048435,0.055145,0.064746,0.064159,0.081355,0.071967,0.075988,0.110050,0.114367


In [124]:
#Se guarda el dataset unificado . Este dataset se usó únicamente para realizar las primeras pruebas del proyecto

df_combined.to_csv(ruta+"dfCombinado.csv", index=False)

**2 Creación de dataset final donde cada fila se corresponde con el par empresa-año**

Se reestructura el dataset con todas las variables integradas de manera que cada fila estará identificada por el par (ID de empresa, año).

Para cada fila solo habra una columna de cada variable, referida a una empresa y un año en concreto. (Por ejemplo, en lugar de PT2010...PT2022, solo hay una variable PT)


In [125]:
# Para este análisis se descartan y se borran los atibutos correrspondientes a los indices de zombificacion por tres años

cols_to_drop = [col for col in df_combined.columns if "-20" in col]


df_combined.drop(cols_to_drop, axis=1, inplace=True)




# Se reorganiza el dataset para incluir el año como una columna nueva

# La función melt te permite mantener el ID y reorganizar todo lo demás
df_melted = pd.melt(df_combined, id_vars=['ID'], var_name='variable', value_name='valor')

# Extraemos la medida y el año de la columna 'variable'
df_melted['medida'] = df_melted['variable'].str.extract(r'([a-zA-Z]+)\d+$')
df_melted['año'] = df_melted['variable'].str.extract(r'(\d{4})$')

# Ahora podemos reorganizar el dataframe para que tenga las columnas deseadas
df_final = df_melted.pivot_table(index=['ID', 'año'], columns='medida', values='valor', aggfunc='first').reset_index()

# Renombrar las columnas para quitar el nombre multi-índice generado por pivot_table
df_final.columns.name = None
df_final = df_final.rename_axis(None, axis=1)

# La nueva columna año se normaliza
scaler = MinMaxScaler(feature_range=(0, 1))
df_final['año'] = scaler.fit_transform(df_final[['año']])

In [126]:
df_final = pd.merge(df_final, df_datosEmpresa, on="ID", how="inner")

In [127]:
df_final

,ID,año,AC,Endeudamiento,FondoManiobra,PT,R,RAGXFI,RAIE,Solvencia,Zombi,SECTOR,RANGO_INGRESOS,EMPLEADOS,EDAD_EMPRESA,TIPOSOC
0,2.331510e+05,0.000000,0.023913,0.453292,0.476216,0.358907,0.476216,0.001411,0.040485,0.080816,0.0,0.352941,0.1,0.038114,0.460784,1.0
1,2.331510e+05,0.083333,0.026300,0.463274,0.471618,0.355629,0.471618,0.001411,0.043602,0.064498,0.0,0.352941,0.1,0.038114,0.460784,1.0
2,2.331510e+05,0.166667,0.030259,0.488989,0.473764,0.355992,0.473764,0.001411,0.052536,0.059908,0.0,0.352941,0.1,0.038114,0.460784,1.0
3,2.331510e+05,0.250000,0.038109,0.507304,0.499606,0.359846,0.499606,0.001411,0.056812,0.059818,0.0,0.352941,0.1,0.038114,0.460784,1.0
4,2.331510e+05,0.333333,0.022514,0.481316,0.475650,0.360398,0.475650,0.007388,0.046632,0.103046,1.0,0.352941,0.1,0.038114,0.460784,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159453,2.121895e+10,0.666667,0.063454,0.424169,0.596856,0.395630,0.596856,0.030379,0.027967,0.187216,0.0,0.647059,0.0,0.008024,0.107843,1.0
159454,2.121895e+10,0.750000,0.070663,0.425968,0.554215,0.422019,0.554215,0.030379,0.013401,0.202884,0.0,0.647059,0.0,0.008024,0.107843,1.0
159455,2.121895e+10,0.833333,0.076286,0.425156,0.535840,0.460604,0.535840,0.030379,0.013814,0.215780,0.0,0.647059,0.0,0.008024,0.107843,1.0
159456,2.121895e+10,0.916667,0.092759,0.421266,0.652722,0.517823,0.652722,0.030379,0.025238,0.642806,0.0,0.647059,0.0,0.008024,0.107843,1.0


In [128]:
df_final.info() #se comprueba que no hay valores nulos

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159458 entries, 0 to 159457
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              159458 non-null  float64
 1   año             159458 non-null  float64
 2   AC              159458 non-null  float64
 3   Endeudamiento   159458 non-null  float64
 4   FondoManiobra   159458 non-null  float64
 5   PT              159458 non-null  float64
 6   R               159458 non-null  float64
 7   RAGXFI          159458 non-null  float64
 8   RAIE            159458 non-null  float64
 9   Solvencia       159458 non-null  float64
 10  Zombi           159458 non-null  float64
 11  SECTOR          159458 non-null  float64
 12  RANGO_INGRESOS  159458 non-null  float64
 13  EMPLEADOS       159458 non-null  float64
 14  EDAD_EMPRESA    159458 non-null  float64
 15  TIPOSOC         159458 non-null  float64
dtypes: float64(16)
memory usage: 20.7 MB


In [129]:
df_final.isna().sum()

ID                0
año               0
AC                0
Endeudamiento     0
FondoManiobra     0
PT                0
R                 0
RAGXFI            0
RAIE              0
Solvencia         0
Zombi             0
SECTOR            0
RANGO_INGRESOS    0
EMPLEADOS         0
EDAD_EMPRESA      0
TIPOSOC           0
dtype: int64

In [130]:
df_final.to_csv(ruta+"dfFinal.csv", index=False) #se guarda el dataset integrado en un fichero csv.